`pidgin.weave` defines how the input source with be displayed, and always follows `pidgin.tangle`. 

Weaving places `display` as having the same importance as `eval` and `exec` 

lexical analysis i sperforme don non python source.


In [1]:
    import IPython, re, functools, typing as t
    with __import__('importnb').Notebook():
        from . import tangle
    with tangle.Pidgin():
        
        from . import string_formatter, environment
    if __name__ == '__main__':
        %reload_ext pidgin
        ip = get_ipython()
        import pidgin

ImportError: cannot import name 'tangle'

In [1]:
The weave phase is trigger by the upython display

https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html
    

SyntaxError: invalid syntax (<ipython-input-1-af3c99307001>, line 1)

### Implicit and Explicit Documentation.

Notebooks make documentation.

In linear programming, both the code and narrative provide explicit directives to style the docuemntation

Documentation should be run as tests -nbsphinx

In [21]:
    def load_ipython_extension(ip: IPython.core.interactiveshell.InteractiveShell)->None:
        with __import__('importnb').Notebook(): from . import string_formatter
        string_formatter.load_ipython_extension(ip)
        ip.weave = Weave(parent=ip)
        ip.events.register('post_run_cell', weave)

    def unload_ipython_extension(ip: IPython.core.interactiveshell.InteractiveShell)->None:
        ip.events.callbacks['post_run_cell'] = [
            object for object in ip.events.callbacks['post_run_cell'] if object.__name__ != 'weave']
        with __import__('importnb').Notebook(): from . import string_formatter
        string_formatter.unload_ipython_extension(ip)

In [22]:
    def weave(result) -> t.Tuple[dict, dict]: return IPython.get_ipython().weave.format(result.info.raw_cell)

`pidgin.weave` establishes __Markdown__ rules for displaying cell inputs after the `ip.run_cell` has been evaluated.

`pidgin.weave` is a secondary literate program with rules for displaying input encoded in `mistune.markdown`.

In [23]:
    import re, dataclasses, ast, re,traceback, functools, IPython, jinja2, inspect, slugify, collections, html, fnmatch, mistune, inspect, importlib, types, base64, string, traitlets, yaml, io, textwrap
    import nbconvert
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    ip = IPython.get_ipython()

    with __import__('importnb').Notebook():
        from . import doctesting, tangle


> The following regular expressions are copied from `mistune`.

`mistune.BlockGrammar and mistune.InlineGrammar` includes doctype and prolog parsing in the html and should be used with `mistune.BlockLexer`.

In [24]:
    class InlineHtmlGrammar:
        """Add svg constructs to inline_html regex."""
        inline_html = re.compile(
                r'^(?:%s|%s|%s)' % (
                    r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                    r'<(\w+%s)((?:%s)*?)\s*>([\s\S]*?)<\/\1>' % (
                        mistune._valid_end, mistune._valid_attr),
                    r'<\w+%s(?:%s)*?\s*\/?>' % (mistune._valid_end, mistune._valid_attr)))
        
    class InlineGrammar(InlineHtmlGrammar, mistune.InlineGrammar): ...

In [25]:
    error = "background-color: var(--jp-rendermime-error-background);"
    ws = "white-space: pre;"

In [26]:
    def whiten(str: str) -> str: return re.sub('\S{1}', ' ', str)

In [27]:
    class WeaveInlineLexer(mistune.InlineLexer):
        """WeavePidginInlineLexer is responsible for updating modifying the interactive IPython state.
        """
        grammar_class = InlineGrammar

In [28]:
    class WeaveRenderer(mistune.Renderer):
        """WeaveRenderer is responsbile for formatting the output of the input code."""
        failure = None
        tmp_code = []
         
        def block_code(self, text: str, lang=None) -> str:
            style = ws
            if lang is None: 
                if self.failure: style += error
                return F"""<code style="{style}" title="{self.failure or ""}">{text}</code>"""
            return super().block_code(text, lang)
        
        def codespan(self, text: str) -> str:
            # Code can exist across multiple cells.
            text = '\n'.join([*self.tmp_code, text])
            if text.endswith('\\'): 
                self.tmp_code = text.splitlines()
                return super().codespan(text)
            
            result = IPython.get_ipython().run_cell(textwrap.indent(text, '    '), silent=True)

            error = result.error_in_exec or result.error_before_exec
            text = text.splitlines()[-1]                
            if error:
                return F"""<code style="{error}" title="{format_title(error)}">{text}</code>"""
            return F"""<code>{text}</code>"""
        
        def text(self, text: str) -> str: return text
        
        def doctest(self, text: str) -> str:
            if ip and ip.weave.doctest:
                with IPython.utils.capture.capture_output() as out:
                    # variables created in the doctest namespace are stored in a specific table.
                    runner = doctesting.run_docstring_examples(text, IPython.get_ipython(), verbose=True)
            if runner.failures:
                IPython.get_ipython().showtraceback((doctesting.DocTestException, doctesting.DocTestException(out.stdout), BaseException().__traceback__))

            return F"""<code style="{ws + error if runner.failures else ''}" title="{format_title(out.stdout)
                }">{text}</code>"""  
        
        def footnote_ref(self, key, index):
            return F"""<sup class="footnote-ref" id="fnref-{slugify.slugify(key)}">
                <a title="{format_title(IPython.get_ipython().user_ns.get(key, ""))
                }" href="#fn-{mistune.escape(key)}">{key}</a></sup>"""
        
        
        def link(self, link, title, text):
            object = super().link(link, title, text)
            IPython.get_ipython().user_ns.update({link: link})
            if title:
                IPython.get_ipython().user_ns.update({title: object})
            return object

        def image(self, src, title, text):
            IPython.get_ipython().user_ns.update({src: IPython.display.Image(src)})
            return super().image(src, title, text)

In [29]:
    link = """[{text}]({link} "{title}")"""

In [30]:
    class WeaveRenderer(mistune.Renderer):
        """WeaveRenderer is responsbile for formatting the output of the input code."""
        failure = None
        tmp_code = []
         
        def codespan(self, text: str) -> str:
            all = '\n'.join([*self.tmp_code, text])
            if all.endswith('\\'): 
                self.tmp_code = all.splitlines()
                return super().codespan(text)
            self.tmp_code = []
            text = string_formatter.environment.ipython_output(F"`{all}`")
            return super().codespan(text)
        
        def text(self, text: str) -> str: return text
        
        def doctest(self, text: str) -> str:
            text = string_formatter.environment.ipython_output(text)
            return super().block_code(text)
        
        def link(self, link, title, text):
            string_formatter.environment.ipython_output(link.format(**locals()))
            return super().link(link, title, text)

        def image(self, link, title, text):
            string_formatter.environment.ipython_output('!'+link.format(**locals()))
            return super().image(link, title, text)

add header and footnotes

In [31]:
User statements versus user expressions.

User statements versus user expressions.

In [32]:
    class WeaveMarkdown(mistune.Markdown):
        """WeaveMarkdown combines initializes and applies WeavePidginBlockLexer and WeavePidginInlineLexer.
        """
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def render(self, text, failure=None):
            self.renderer.failure = failure
            return super().render(text)
        
        def parse(self, text):
            out = self.output(mistune.preprocessing(text))
            keys = self.block.def_footnotes
            self.block.def_footnotes = {}
            self.inline.footnotes = {}
            
            if not self.footnotes: return out

            self.footnotes = sorted(self.footnotes, key=lambda o: keys.get(o['key']), reverse=True)

            body = self.renderer.placeholder()
            while self.footnotes:
                note = self.footnotes.pop()
                body += self.renderer.footnote_item(note['key'], note['text'])

            out += self.renderer.footnotes(body)
            return out

In [33]:
    class BlockLexer(tangle.BlockLexer):
        def __init__(self, rules=None, **kwargs):
            super().__init__(rules, **kwargs)
            self.def_links = IPython.get_ipython().weave.def_links

In [34]:
    def markdown() -> mistune.Markdown: 
        return WeaveMarkdown(
        renderer=WeaveRenderer(escape=False, parse_block_html=True, parse_inline_html=False),
        block=BlockLexer, inline=WeaveInlineLexer)

In [35]:
    def do_publish(data: str) -> bool:
        is_display = bool(data.partition('\n')[0].strip() and not data.startswith(';')) 
        source = tangle.markdown_to_python(data)
        is_display = is_display & ~bool(only_contains_block_code(data, source))
        try:
            if data.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): is_display = False
        except: ...
        return is_display

In [36]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return raw.startswith('%%') or (
            list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines())))

In [37]:
    ip.Completer.use_jedi=False

In [38]:
    class Weave(IPython.core.formatters.IPythonDisplayFormatter):
        """`Weave` modifies the __weave__ phase of the cell execution."""

        safe = traitlets.Bool(True, help="""Replace file paths with testable and safe strings. """)
        html = traitlets.Bool(True, help="""Return an annotated and custom html view for the markdown input.""")        
        doctest = traitlets.Bool(True, help="""Evaluate doctests in the source..""")        
        inline = traitlets.Bool(True, help="""Evaluate inline code elements..""")
        def_links = traitlets.Dict(default_value={})
        
        def front_matter(self, str):
            if str.startswith('---'):
                if '\n---' in str:
                    import yaml, io
                    body = str.lstrip('---')
                    data, sep, body = body.partition('---\n')
                    data = yaml.safe_load(
                        io.StringIO(self.environment.from_string(data).render()))
                    if isinstance(data, dict): return body, data
            return str, {}

        def format(self, raw, include=None, exclude=None, *, body="", exc=None):
            with self:
                source, metadata = self.front_matter(raw)
                source = self.template(source, **metadata)
                data, _metadata = self.parent.display_formatter.format(source)
                lines = source.splitlines(True)        
                body = markdown().render(''.join(lines))
                if self.html: data['text/html'] = data.get('text/html', body)
                if do_publish(raw): IPython.get_ipython().display_pub.publish(data, {**metadata, **_metadata})
                
        """The parent is the displayformatter"""
        
        environment = traitlets.Instance(jinja2.Environment)
        
        @traitlets.default('environment')
        def _default_environment(self): return environment.new_environment()
        
        level = traitlets.Int(default_value=0)
        def __enter__(cls): cls.level += 1
        def __exit__(cls, *exc): cls.level -= 1

        def template(self, object, **data): 
            if self.level == 1: 
                object = self.environment.from_string(object).render(
                    collections.ChainMap(data, self.parent.user_ns))
            return object

In [39]:
    def format_title(object): 
        return '&#013;'.join(html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [27]:
The weave step is similar to the user_expressions.  Only evaluating expressions was too limiting.

The weave step is similar to the user_expressions.  Only evaluating expressions was too limiting.

`run_cell_display` triggers a display on the input if contains anything other than block code.

In [28]:
    if __name__ == '__main__':
        unload_ipython_extension(get_ipython())
        tangle.unload_ipython_extension(get_ipython())
        tangle.load_ipython_extension(get_ipython())
        load_ipython_extension(get_ipython())